# Human Activity Recognition - EDA & Hidden Data Issue Detection 
**Mục tiêu:** Tìm ra vấn đề ẩn trong dataset cảm biến (6-axis accelerometer tay & chân)
## Load & Quan sát cấu trúc file

In [ ]:
import pandas as pd
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Load all files
files = glob.glob("*.csv")
print(f"Tìm thấy {len(files)} files:")
for f in files:
    df_temp = pd.read_csv(f, nrows=1)
    print(f"{f:20} → columns: {list(df_temp.columns)} → shape: {pd.read_csv(f).shape}")

# Các file *_2.csv có thứ tự cột đảo ngược
Chú ý header: có file bắt đầu bằng **arm**, có file bắt đầu bằng **leg**

In [ ]:
# Hàm đọc file và tự động sửa thứ tự cột dựa trên phân bố thực tế
def smart_read_csv(filepath):
    df = pd.read_csv(filepath)
    
    # Nếu có cột label → bỏ tạm để phân tích
    if 'label' in df.columns:
        label = df['label'].iloc[0]
        df = df.drop('label', axis=1)
    else:
        label = None
    
    cols = df.columns.tolist()
    
    # Heuristic: kênh nào có biên độ lớn hơn, dao động nhiều hơn → thường là leg
    # Tay thường có giá trị thấp hơn
    leg_candidates = [c for c in cols if 'leg' in c.lower()]
    arm_candidates = [c for c in cols if 'arm' in c.lower()]
    
    if len(leg_candidates) == 3 and len(arm_candidates) == 3:
        # Dựa trên độ lệch chuẩn (std) của trục z để xác định
        std_leg_z = df[leg_candidates[2]].std()
        std_arm_z = df[arm_candidates[2]].std()
        
        # Nếu "leg" lại có std nhỏ hơn "arm" → chắc chắn bị đảo
        if std_leg_z < std_arm_z:
            print(f"PHÁT HIỆN ĐẢO KÊNH ở file: {filepath.split('/')[-1]}")
            # Đảo lại cho đúng
            correct_order = arm_candidates + leg_candidates
        else:
            correct_order = leg_candidates + arm_candidates
    else:
        # Fallback: dựa vào tên file có "_2" để xác định
        if "_2" in filepath:
            correct_order = cols[3:] + cols[:3]  # đảo ngược
            print(f"File _2 → tự động đảo cột: {filepath.split('/')[-1]}")
        else:
            correct_order = cols
    
    df_corrected = df[correct_order]
    if label is not None:
        df_corrected['label'] = label
    
    df_corrected.columns = ['leg_acc_x','leg_acc_y','leg_acc_z','arm_acc_x','arm_acc_y','arm_acc_z','label']
    return df_corrected

# Đọc lại toàn bộ và sửa lỗi
dfs = []
for f in files:
    if f.endswith(".csv"):
        df = smart_read_csv(f)
        df['source_file'] = f
        dfs.append(df)

data = pd.concat(dfs, ignore_index=True)
print(f"\nTổng dữ liệu sau khi chuẩn hóa: {data.shape}")

# Trực quan: Phân bố trước và sau khi sửa

In [ ]:
# So sánh phân bố của leg_z và arm_z trước/sau
fig, axes = plt.subplots(2, 2, figsize=(12,8))

# Trước khi sửa
data_wrong = pd.concat([pd.read_csv(f) for f in files if f.endswith(".csv")], ignore_index=True)
if 'label' in data_wrong.columns:
    data_wrong = data_wrong.drop('label', axis=1, errors='ignore')

sns.kdeplot(data_wrong.iloc[:,2], label='Cột 3 (giả định leg_z)', ax=axes[0,0])
sns.kdeplot(data_wrong.iloc[:,5], label='Cột 6 (giả định arm_z)', ax=axes[0,0])
axes[0,0].set_title("TRƯỚC KHI SỬA")
axes[0,0].legend()

# Sau khi sửa
sns.kdeplot(data['leg_acc_z'], label='leg_acc_z (đã chuẩn hóa)', ax=axes[0,1], color='red')
sns.kdeplot(data['arm_acc_z'], label='arm_acc_z (đã chuẩn hóa)', ax=axes[0,1], color='blue')
axes[0,1].set_title("SAU KHI SỬA")
axes[0,1].legend()

# Boxplot độ lệch chuẩn trục Z theo file
stds = []
for f in files:
    df = pd.read_csv(f)
    if 'label' in df.columns: df = df.drop('label', axis=1)
    stds.append({
        'file': f,
        'std_leg_z': df.iloc[:, df.columns.str.contains('leg').tolist()[2] if any(df.columns.str.contains('leg')) else df.iloc[:,2]].std(),
        'std_arm_z': df.iloc[:, df.columns.str.contains('arm').tolist()[2] if any(df.columns.str.contains('arm')) else df.iloc[:,5]].std()
    })
std_df = pd.DataFrame(stds)

std_df.plot(kind='bar', x='file', y=['std_leg_z', 'std_arm_z'], ax=axes[1,0])
axes[1,0].set_title("Độ lệch chuẩn trục Z theo file → thấy rõ đảo ngược")

plt.tight_layout()
plt.show()

## Executive Summary

- Dataset gồm dữ liệu accelerometer từ 2 subject, mỗi subject có nhiều file với các hoạt động: `jogging`, `jogging (sidesteps)`
- Phát hiện nghiêm trọng: Một số file có thứ tự cột arm/leg bị đảo ngược hoàn toàn nhưng header không đồng nhất
- Bằng chứng: Phân bố giá trị arm vs leg hoàn toàn đảo ngược
- Hậu quả: Mô hình sẽ học sai khái niệm "tay" và "chân" → accuracy giả cao nhưng thực tế fail khi deploy
- Đề xuất: Chuẩn hóa thứ tự cột (dựa trên giá trị thực tế, thay vì header)